In [1]:
from __future__ import annotations
from typing import Self

import random
from math import sqrt
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import phik
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

## 1

> 1. Для студентов с четным порядковым номером в группе – датасет с
>  [классификацией грибов](https://archive.ics.uci.edu/ml/datasets/Mushroom), а
>  нечетным – [датасет с данными про оценки студентов инженерного и педагогического факультетов](https://archive.ics.uci.edu/dataset/856/higher+education+students+performance+evaluation) 
>  (для данного датасета нужно ввести метрику: студент успешный/неуспешный на
>  основании грейда)


У меня 9 номер в группе, поэтому буру датасет про оценки.

In [3]:
ds = pd.read_csv("../res/higher-education-students-performance-evaluation.csv")
ds

,STUDENT ID,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,COURSE ID,GRADE
0,STUDENT1,2,2,3,3,1,2,2,1,1,...,1,1,3,2,1,2,1,1,1,1
1,STUDENT2,2,2,3,3,1,2,2,1,1,...,1,1,3,2,3,2,2,3,1,1
2,STUDENT3,2,2,2,3,2,2,2,2,4,...,1,1,2,2,1,1,2,2,1,1
3,STUDENT4,1,1,1,3,1,2,1,2,1,...,1,2,3,2,2,1,3,2,1,1
4,STUDENT5,2,2,1,3,2,2,1,3,1,...,2,1,2,2,2,1,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,STUDENT141,2,1,2,3,1,1,2,1,1,...,1,1,2,1,2,1,3,3,9,5
141,STUDENT142,1,1,2,4,2,2,2,1,4,...,1,1,3,2,2,1,5,3,9,5
142,STUDENT143,1,1,1,4,2,2,2,1,1,...,1,1,3,3,2,1,4,3,9,1
143,STUDENT144,2,1,2,4,1,1,1,5,2,...,2,1,2,1,2,1,5,3,9,4


In [4]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STUDENT ID  145 non-null    object
 1   1           145 non-null    int64 
 2   2           145 non-null    int64 
 3   3           145 non-null    int64 
 4   4           145 non-null    int64 
 5   5           145 non-null    int64 
 6   6           145 non-null    int64 
 7   7           145 non-null    int64 
 8   8           145 non-null    int64 
 9   9           145 non-null    int64 
 10  10          145 non-null    int64 
 11  11          145 non-null    int64 
 12  12          145 non-null    int64 
 13  13          145 non-null    int64 
 14  14          145 non-null    int64 
 15  15          145 non-null    int64 
 16  16          145 non-null    int64 
 17  17          145 non-null    int64 
 18  18          145 non-null    int64 
 19  19          145 non-null    int64 
 20  20        

In [5]:
ds.describe()

,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,COURSE ID,GRADE
count,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,...,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000,145.000000
mean,1.620690,1.600000,1.944828,3.572414,1.662069,1.600000,1.579310,1.627586,1.620690,1.731034,...,1.337931,1.165517,2.544828,2.055172,2.393103,1.806897,3.124138,2.724138,4.131034,3.227586
std,0.613154,0.491596,0.537216,0.805750,0.474644,0.491596,0.495381,1.020245,1.061112,0.783999,...,0.614870,0.408483,0.564940,0.674736,0.604343,0.810492,1.301083,0.916536,3.260145,2.197678
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000
50%,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,...,1.000000,1.000000,3.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000
75%,2.000000,2.000000,2.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,1.000000,3.000000,3.000000,3.000000,2.000000,4.000000,3.000000,7.000000,5.000000
max,3.000000,2.000000,3.000000,5.000000,2.000000,2.000000,2.000000,5.000000,4.000000,4.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,5.000000,4.000000,9.000000,7.000000


Видно, что данные распознаны и дополнительная обраобтка не нужна.

Введем метрику "успешности" студента на основании атрибута 'GRADE'. Будем считать, что больше среднего значения получает именно успешный студент.

In [6]:
grade_treshold = (np.min(ds['GRADE']) + np.max(ds['GRADE'])) / 2
ds['SUCC'] = (ds['GRADE'] > grade_treshold).astype(int)
ds.sample(10)

,STUDENT ID,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,COURSE ID,GRADE,SUCC
77,STUDENT78,1,2,1,2,2,2,1,2,2,...,1,2,2,2,2,1,1,4,7,1
126,STUDENT127,1,1,1,4,1,1,1,3,2,...,1,3,3,3,1,2,2,9,3,0
73,STUDENT74,2,2,2,4,2,2,2,1,1,...,2,3,2,3,1,5,3,3,6,1
29,STUDENT30,2,2,3,4,2,2,2,1,4,...,1,3,2,2,2,4,3,1,5,1
0,STUDENT1,2,2,3,3,1,2,2,1,1,...,1,3,2,1,2,1,1,1,1,0
114,STUDENT115,2,1,2,5,1,1,2,4,1,...,1,2,2,1,2,1,1,8,2,0
84,STUDENT85,3,2,3,3,1,2,1,3,1,...,3,3,3,3,3,5,4,5,7,1
106,STUDENT107,1,2,2,4,2,1,2,1,1,...,2,2,2,2,1,4,4,7,7,1
35,STUDENT36,1,2,1,4,2,2,2,3,1,...,1,2,1,3,1,1,1,1,1,0
139,STUDENT140,1,1,2,4,1,2,1,2,2,...,1,2,3,2,1,1,2,9,0,0


# 2

> 2. Отобрать **случайным** образом sqrt(n) признаков

In [7]:
features = list(ds.columns.drop(['STUDENT ID', 'GRADE', 'SUCC']))
target = 'SUCC'
print(f"{features=}")
print(f"{target=}")

features=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', 'COURSE ID']
target='SUCC'


In [8]:
fcs_size = int(sqrt(len(features))) # required sqrt(n) number of feature columns
fcs = [features[i] for i in sorted(random.sample(range(len(features)), fcs_size))]
print(f"{fcs=}") # tuple of features in original order
print(f"{fcs_size=}")

fcs=['2', '6', '8', '17', '28']
fcs_size=5


# 3

> 3. Реализовать без использования сторонних библиотек построение дерева решений
>    (дерево не бинарное, numpy и pandas использовать можно, использовать список
>    списков  для реализации  дерева - нельзя)


Реализуем виды узлов в дереве с помощью ООП:

In [40]:
from abc import ABC
from typing import TypeVar

F = TypeVar("F") # feature generic type
V = TypeVar("V") # feature available values generic type
T = TypeVar("T") # target class of prediction: (0, 1) in ds['SUCC']

class DTNode(ABC): 
    """Decision tree abstract node class"""
    INNER = 'inner'
    LEAF = 'leaf'
    def __init__(self, type: str) -> None:
        super().__init__()
        self.nodetype: str = type


class DTInnerNode(DTNode):
    """Decision tree inner node which have some children"""

    def __init__(self, 
                 feature: F, # feature used for split in this node
                 threasholds: list[F], # list of available thresholds
                 children: list[DTNode], # list of children corresponding to list of threasholds
                 gain: float) -> None: # information gain of split in this node
        super().__init__(DTNode.INNER)
        self.feature = feature
        self.threasholds = threasholds
        self.children = children
        self.gain = gain
    

class DTLeafNode(DTNode):
    """Decision tree leaf node which have no children"""

    def __init__(self, value: T) -> None: # value of terminal node in tree
        super().__init__(DTNode.LEAF)
        self.value = value


Для выполнения задания достаточно реализовать дерево решений по *категориальным* признакам (оценки по предметам, по факту *численные дискретные*, но не суть) для *бинарно* признака успешности студента.

In [48]:
class DecisionTreeClassifier():
    def __init__(self, min_samples: int=2, max_depth: int=2) -> None:
        assert min_samples > 0 and max_depth > 0, "parameters can't be negative or zero"
        self.min_samples = min_samples
        self.max_depth = max_depth
    
    def fit(self, X, y) -> None:
        self._root = self._build_tree(X, y)
    
    def predict(self, X) -> pd.Series:
        return pd.Series([ self._predict(x) for i, x in X.iterrows() ])
    
    def _build_tree(self, X, y) -> DTNode:
        curr_depth = 1
        return self._build_sub_tree(X, y, curr_depth)
    
    def _build_sub_tree(self, X, y, curr_depth) -> DTNode:
        if len(X) > self.min_samples and curr_depth < self.max_depth:
            feature, thresholds, gain, children_datasets = self._best_split(X)

            if gain > 0: # gain can be 0 in worst case
                children = [self._build_sub_tree(X_child, y_child, curr_depth + 1)
                            for X_child, y_child in children_datasets]
                return DTInnerNode(feature, thresholds, gain, children)
        
        return DTLeafNode(self._leaf_value(y))


    def _predict(self, x, node: DTNode) -> T:
        if node.nodetype == DTNode.LEAF:
            return node.value

        node: DTInnerNode 

        for i, t in enumerate(node.threasholds):
            if x[node.feature] == t:
                return self._predict(x, node.children[i])

        raise AssertionError("Error in tree structure: at least one feature must"
                             " be used in node (may be train and prediction data" 
                             " formats differ)")

    def _leaf_value(self, y: pd.Series) -> T:
        return y.value_counts().index[0]

    def _best_split(self, X, y) -> tuple[F, list[V[F]], float, list[pd.DataFrame]]:
        """
        returns tuple of feature name used for best split, 
                available values of this feature (aka thresholds), 
                information gain after split 
                and list of children's datasets for such thresholds
        """
        best_info_gain = -1
        best_feature = None
        best_thresholds = None
        best_children_datasets = None

        for fname in X.columns:
            thresholds = sorted(pd.unique(X[fname]))
            # feature_tuples = [X[fname] == t for t in thresholds]
            # children_datasets = [(X[fts], y[fts]) for fts in feature_tuples]
            children_datasets = [(X[fts], y[fts])
                                 for t in thresholds 
                                 if (fts := (X[fname] == t))]

            if len(children_datasets) < 2:
                continue
            
            info_gain = self._info_gain(y, [y for X, y in children_datasets])

            if info_gain > best_info_gain:
                best_gain = info_gain
                best_feature = fname
                best_thresholds = thresholds
                best_children_datasets = children_datasets

        return best_feature, best_thresholds, best_gain, best_children_datasets

    def _info_gain(self, y_parent, y_children) -> float:
        parent_entropy = self._entropy(y_parent)

        weighted_entropy = 0
        for y_child in y_children:
            weight = len(y_child) / len(y_parent)
            weighted_entropy += weight * self._entropy(y_child)
        
        return parent_entropy - weighted_entropy


    def _entropy(self, y) -> float:
        _, counts = np.unique(y, return_counts=True)
        prob = counts / len(y)
        entropies = -prob * np.log2(prob)

        # entropy = 0
        # for value in pd.unique(y):
        #     vprob = np.count_nonzero(y == value) / len(y)
        #     entropy += -vprob * np.log2(vprob)
        
        return np.sum(entropies)


In [11]:
class DecisionTree():
    def __init__(self, min_samples: int=2, max_depth: int=2) -> None:
        assert min_samples > 0 and max_depth > 0
        self.min_samples = min_samples
        self.max_depth = max_depth
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> Self:
        dataset = np.concatenate((X, np.expand_dims(y, axis=-1)), axis=1)
        self.root = self._build_tree(dataset)
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        predictions = [self._make_prediction(x, self.root) for x in X.values]

        return np.array(predictions)

    def _make_prediction(self, x: np.ndarray, node: DTNode) -> float:
        if node.value is not None:
            return node.value

        for i, t in enumerate(node.thresholds):
            if x[node.feature] == t: # works for categorial features
                return self._make_prediction(x, node.children[i])

        return self._make_prediction(x, random.choice(node.children))

    def _entropy(self, y: pd.Series) -> float:
        entropy = 0
        labels = np.unique(y)
        for label in labels:
            lprob = np.count_nonzero(y == label) / len(y)
            entropy += -lprob * np.log2(lprob)

        return entropy
    
    def _information_gain(self, parent: pd.Series, children: pd.DataFrame):
        parent_entropy = self._entropy(parent)
        weighted_entropy = 0

        for child in children:
            weight = len(child) / len(parent)
            weighted_entropy += weight * self._entropy(child)

        return parent_entropy - weighted_entropy
    
    def _split_data(self, dataset, feature, thresholds):
        datasets = [dataset[dataset[:, feature] == t] for t in thresholds]

        return datasets
    
    def _best_split(self, dataset: np.ndarray):
        num_features = dataset.shape[1] - 1

        best_split = {'gain': -1, 'feature': None, 'threshold': None}

        for feature_index in range(num_features):
            thresholds = sorted(np.unique(dataset[:, feature_index]))
            children = self._split_data(dataset, feature_index, thresholds)

            if len(children) < 2:
                continue

            children_y = [child[:, -1] for child in children]

            information_gain = self._information_gain(dataset[:, -1], children_y)

            if information_gain > best_split["gain"]:
                best_split["feature"] = feature_index
                best_split['thresholds'] = thresholds
                best_split["children"] = children
                best_split["gain"] = information_gain

        return best_split
    
    def _calculate_leaf_value(self, y):
        return Counter(y).most_common(1)[0][0]

    def _build_tree(self, dataset: np.ndarray, curr_depth: int=0) -> DTNode:
        X, y = dataset[:, :-1], dataset[:, -1]

        if X.shape[0] >= self.min_samples and curr_depth < self.max_depth:
            best_split = self._best_split(dataset)

            if best_split["gain"] > 0:
                children_nodes = []

                for child_dataset in best_split['children']:
                    next_node = self._build_tree(child_dataset, curr_depth + 1)
                    children_nodes.append(next_node)

                return DTNode(
                    feature=best_split["feature"], 
                    thresholds=best_split["thresholds"],
                    children=children_nodes, 
                    gain=best_split["gain"]
                )

        return DTNode(value=self._calculate_leaf_value(y))

In [12]:

ds_enc = 
X_train, X_test, y_train, y_test = train_test_split(ds[fcs], ds[target], test_size=0.3, random_state=42)
model = DecisionTree(max_depth=5).fit(X_train, y_train)


SyntaxError: invalid syntax (888647390.py, line 1)

In [ ]:
y_pred = model.predict(X_test)
y_pred

array([1, 1, 1, 1, 3, 1, 7, 3, 5, 1, 7, 5, 1, 1, 1, 3, 6, 3, 5, 5, 5, 7,
       3, 1, 1, 1, 3, 1, 1, 1, 1, 5, 4, 3, 5, 1, 5, 3, 7, 1, 7, 5, 2, 5],
      dtype=int64)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
mm = DecisionTreeClassifier(min_samples_leaf=2,max_depth=5).fit(X_train, y_train)
y_pred_mm = mm.predict(X_test)
roc_auc_score(y_test, y_pred, multi_class='ovo')#, roc_auc_score(y_test, y_pred_mm)

AxisError: axis 1 is out of bounds for array of dimension 1

4. Провести оценку реализованного алгоритма с использованием Accuracy, precision и recall


5. Построить кривые AUC-ROC и AUC-PR 